In [1]:
import subprocess
import os
import csv
import pandas as pd
from tqdm import tqdm
path = 'C:/Users/6997619/Documents/GitHub projects/DHZW-simulation_Sim-2APL/src/main/resources/sensitivity_analysis/'

# Initialise

### Define the parameters and the values to try

In [2]:
# Open the CSV file and read it as a list of dictionaries
with open(path + 'data/parameter_labels.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    params = [row['parameter'] for row in reader]

In [3]:
# Define the range of values for each parameter
values = [0.4, 0.7, 1.0, 1.3, 1.6] # -60%, -30%, default, 30%, 60%

### Save the parameter sets in file

In [4]:
# Create an empty table with the column headers from params
table = pd.DataFrame(columns=params)

table['used_parameter_label'] = 0
table['used_parameter_value'] = 0

table

,alphaWorkCarDriver,alphaWorkCarPassenger,alphaWorkBus,alphaWorkTrain,betaTimeWorkWalk,betaTimeWorkBike,betaTimeWorkCarDriver,betaTimeWorkCarPassenger,betaTimeWorkBus,betaTimeWorkTrain,...,betaCostLeisureBus,betaCostLeisureTrain,betaTimeWalkTransportWork,betaTimeWalkTransportSchool,betaTimeWalkTransportLeisure,betaChangesTransportWork,betaChangesTransportSchool,betaChangesTransportLeisure,used_parameter_label,used_parameter_value


In [5]:
import warnings
warnings.filterwarnings('ignore')
for param in params:
    for value in values:
        # Create a row with all zeros except for the current value of the loop
        zeros_row = pd.Series([1.0 if p != param else value for p in params], index=params)
        zeros_row['used_parameter_label'] = param
        zeros_row['used_parameter_value'] = value
        table = table.append(zeros_row, ignore_index=True)
        
warnings.filterwarnings('default')

In [6]:
table

,alphaWorkCarDriver,alphaWorkCarPassenger,alphaWorkBus,alphaWorkTrain,betaTimeWorkWalk,betaTimeWorkBike,betaTimeWorkCarDriver,betaTimeWorkCarPassenger,betaTimeWorkBus,betaTimeWorkTrain,...,betaCostLeisureBus,betaCostLeisureTrain,betaTimeWalkTransportWork,betaTimeWalkTransportSchool,betaTimeWalkTransportLeisure,betaChangesTransportWork,betaChangesTransportSchool,betaChangesTransportLeisure,used_parameter_label,used_parameter_value
0,0.4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,alphaWorkCarDriver,0.4
1,0.7,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,alphaWorkCarDriver,0.7
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,alphaWorkCarDriver,1.0
3,1.3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,alphaWorkCarDriver,1.3
4,1.6,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,alphaWorkCarDriver,1.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.4,betaChangesTransportLeisure,0.4
236,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.7,betaChangesTransportLeisure,0.7
237,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,betaChangesTransportLeisure,1.0
238,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.3,betaChangesTransportLeisure,1.3


In [7]:
#Save the table
table.to_csv(path + 'data/parametersets.csv', index=False)

### Initialise the output dataframe

In [11]:
df = pd.DataFrame(columns=["CAR_DRIVER", "CAR_PASSENGER", "BIKE", "BUS_TRAM", "TRAIN", "WALK"])

for i in range(0, len(table)):
    df.to_csv(path + "output/output_proportions_" + str(i) + ".csv", index=False)

# Call Sim2APL iteratively

### Prepare the Java command

In [13]:
# baseline of the Java command
config = "--config src/main/resources/config_DHZW_full.toml"
modeliberal = "--mode-liberal 0.5"
modeconservative = "--mode-conservative 0.5"
output = "-o output/deskrun"
parametersPath = "--parameter_file src/main/resources/sensitivity_analysis/data/parametersets.csv"

cmd = "java -cp target/sim2apl-dhzw-simulation-1.0-SNAPSHOT-jar-with-dependencies.jar main.java.nl.uu.iss.ga.Simulation" + " " + config + " " + modeliberal + " " + modeconservative + " " + output + " " + parametersPath

os.chdir('C:/Users/6997619/Documents/GitHub projects/DHZW-simulation_Sim-2APL')

### Main loop to call the simulations

In [24]:
for idx, row in tqdm(table.iterrows(), total=table.shape[0]):
    
    # repeat each parameter set 5 times
    for iteration in range(0, 5):
    
        # output file for this iteration
        outputPath = f'--output_file=src/main/resources/sensitivity_analysis/output/iterations/output_proportions_{idx}.csv'
        full_command = f'{cmd} {outputPath}'

        # parameterset to use
        arg = f'--parameterset_index={idx+1}'
        full_command = f'{cmd} {arg}'
      
        # Capture the output of the Java program
        try:
            output = subprocess.check_output(full_command, stderr=subprocess.STDOUT, universal_newlines=True)
        except subprocess.CalledProcessError as e:
            print(f"Java program exited with non-zero return code: {e.returncode}")
            print(f"Error message: {e.output}")
            exit(1)

100%|██████████████████████████████████████████████████████████████████████████████| 240/240 [00:00<00:00, 8608.85it/s]

test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
